In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import time 

In [5]:
DATASET_PATH = "../input/breast-cancer-wisconsin/breast-cancer-wisconsin.csv"

dataset = pd.read_csv(DATASET_PATH)
print(dataset)

           id  clump  unif_size  unif_shape  marg_adh  secs bare_nucl  \
0     1000025      5          1           1         1     2         1   
1     1002945      5          4           4         5     7        10   
2     1015425      3          1           1         1     2         2   
3     1016277      6          8           8         1     3         4   
4     1017023      4          1           1         3     2         1   
...       ...    ...        ...         ...       ...   ...       ...   
6286   776715      3          1           1         1     3         2   
6287   841769      2          1           1         1     2         1   
6288   888820      5         10          10         3     7         3   
6289   897471      4          8           6         4     3         4   
6290   897471      4          8           8         5     4         5   

      bl_chro  norm_nucl  motises  class  
0           3          1        1      2  
1           3          2        1    

In [46]:
# Preprocessing
# ---------------------------

# dataset.drop(['id'], axis=1, inplace=True)
dataset.replace('?', '0', inplace=True)

for _ in range(3):
    dataset = dataset.sample(frac=1)

# X = dataset[['clump', 'unif_size', 'unif_shape', 'marg_adh', 'secs', 'bare_nucl', 'bl_chro', 'norm_nucl', 'motises' ]]
X = dataset.drop('class', 1)
X = (X.astype(int)) / 10.03125 # 0.03125 scale bias

y = dataset[[ 'class' ]]
y.replace(2, 0, inplace=True)
y.replace(4, 1, inplace=True)

X = np.array(X)
y = np.array(y)

train_X = X[:len(X)-25]
train_y = y[:len(y)-25]

test_X = X[-25:]
test_y = y[-25:]

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.callbacks import TensorBoard

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [47]:
# Keras Model Preparation
# 81x3 feed-forward Model
# ---------------------------

model = Sequential()

# Input Layer
model.add( Dense(27, input_shape=train_X.shape[1:]) )
model.add( Dropout(rate=0.15) )

# Hidden Layer 1
model.add( Dense(81) )
model.add( Activation('relu') )

# Hidden Layer 2
model.add( Dense(81) )
model.add( Activation('relu') )

# Hidden Layer 3
model.add( Dense(27) )
model.add( Activation('relu') )

# Output Layer
model.add( Dense(1) )
model.add( Activation('sigmoid') )

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [48]:
ModelName = "81x3-Keras-" + str(int(time.time()))
print(ModelName)

tensorboard = TensorBoard(log_dir="logs/".format(ModelName))

model.fit(train_X, train_y, batch_size=16, validation_split=0.25, epochs=1, callbacks=[tensorboard])

81x3-Keras-1586041098
Train on 4699 samples, validate on 1567 samples
Epoch 1/3
4699/4699 [==============================] - 1s 311us/sample - loss: 0.1951 - accuracy: 0.9408 - val_loss: 0.0743 - val_accuracy: 0.9681
Epoch 2/3
4699/4699 [==============================] - 1s 178us/sample - loss: 0.0906 - accuracy: 0.9681 - val_loss: 0.0662 - val_accuracy: 0.9706
Epoch 3/3
4699/4699 [==============================] - 1s 179us/sample - loss: 0.0808 - accuracy: 0.9721 - val_loss: 0.0525 - val_accuracy: 0.9802


In [49]:
correct = 0
for k in range(25):
    _y = model.predict(test_X[k].reshape(1,9))
    print("Prediction: " + str(int(_y[0][0]+0.25)) + "  Actual: " + str(test_y[k][0]))
    if(int(_y[0][0]+0.25)==test_y[k][0]):
        correct += 1
        
print()
print("Test Accuracy:")
print((correct/25)*100, '%')

Prediction: 1  Actual: 1
Prediction: 0  Actual: 0
Prediction: 1  Actual: 1
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 1  Actual: 1
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 1  Actual: 1
Prediction: 0  Actual: 0
Prediction: 1  Actual: 1
Prediction: 1  Actual: 1
Prediction: 1  Actual: 1
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0
Prediction: 0  Actual: 0

Test Accuracy:
100.0 %
